# Type et méthodes

*Julia* possède un système de type et de méthode qui lui confère une approche objet.
La fonction `typeof()` renvoie le type d'une variable de base `Int32`, `Float64`... *Julia* est conçu pour permettre facilement d'étendre l'environnement à de nouveaux type de variable.

Le types sont organisés suivant un hiérarchie comme on peut le voir sur l'arborescence partielle ci-dessous


In [1]:
using AbstractTrees

AbstractTrees.children(x::Type) = subtypes(x)

print_tree(Number)

Number
├─ Complex
└─ Real
   ├─ AbstractFloat
   │  ├─ BigFloat
   │  ├─ Float16
   │  ├─ Float32
   │  └─ Float64
   ├─ AbstractIrrational
   │  └─ Irrational
   ├─ Integer
   │  ├─ Bool
   │  ├─ Signed
   │  │  ├─ BigInt
   │  │  ├─ Int128
   │  │  ├─ Int16
   │  │  ├─ Int32
   │  │  ├─ Int64
   │  │  └─ Int8
   │  └─ Unsigned
   │     ├─ UInt128
   │     ├─ UInt16
   │     ├─ UInt32
   │     ├─ UInt64
   │     └─ UInt8
   └─ Rational


Dans cette arborescence, certains types sont "abstraits" et d'autres "concrets".

In [2]:
isconcretetype(Real), isconcretetype(Float64)

(false, true)

Un réel sera forcemment de type concret `Float64` ou `Float32` par exemple, et pourra être utilisé comme argument par toutes les fonctions acceptant le type abstrait `AbstractFloat`

In [6]:
Float64 <: AbstractFloat

true

# Méthodes

A chaque fonction est associée une méthode dépendante du type d'entrée comme dans ce qui suit suivant que l'entrée soit un entier ou pas.

In [8]:
function f(x::Any)
    sin(x+1)
end

f (generic function with 1 method)

In [9]:
function f(n::Integer)
    n
end

f (generic function with 2 methods)

In [12]:
f(3.0)

-0.7568024953079282

In [13]:
methods(f)

# 2 methods for generic function "f":
[1] f(n::Integer) in Main at In[9]:1
[2] f(x) in Main at In[8]:1

In [14]:
f(3)

3

In [15]:
f(im)

1.2984575814159773 + 0.6349639147847361im

In [16]:
f(-2)

-2

In [17]:
+

+ (generic function with 208 methods)

In [18]:
+(1,2)

3

In [19]:
f(sqrt(2))

0.6649143126867011

# Construction d'un nouveau Type de variable

En premier lieu il faut définir un type abstrait puis une instance sous-hiérarchiquement concrète :

In [21]:
abstract type AbstractGrid end # juste en dessous de Any

mutable struct Grid1d <: AbstractGrid
    debut::Float64
    fin::Float64
    n::Int32
end

In [31]:
@show a = Grid1d(0, 1, 2)

a = Grid1d(0, 1, 2) = Grid 1d : début 0.0 , fin 1.0 , 2 éléments



Grid 1d : début 0.0 , fin 1.0 , 2 éléments


In [23]:
a.debut

0.0

In [24]:
a.fin

1.0

In [25]:
a.n

2

# Surcharge des opérateurs

La surcharge des opérations usuelles se fait en définissant une nouvelle méthode associé au nouveau type pour chaque opérateur, commençons par surcharger l'affichage à l'écran de notre nouveau type. Pour cela on va ajouter une méthode à la fonction "show"

In [26]:
function Base.show(io::IO,g::Grid1d)
    print(io, "Grid 1d : début $(g.debut) , fin $(g.fin) , $(g.n) éléments\n")
end

In [27]:
Base.show(a)

Grid 1d : début 0.0 , fin 1.0 , 2 éléments


In [28]:
println(a)

Grid 1d : début 0.0 , fin 1.0 , 2 éléments



In [29]:
a

Grid 1d : début 0.0 , fin 1.0 , 2 éléments


In [30]:
@show a

a = Grid 1d : début 0.0 , fin 1.0 , 2 éléments



Grid 1d : début 0.0 , fin 1.0 , 2 éléments


## Addition, soustraction ...

Ces fonctions sont de la forme +(), -() c'est à dire

In [35]:
import Base:+

function +(g::Grid1d, n::Integer)
    g.n += n
    return g
end

+ (generic function with 209 methods)

In [36]:
a = Grid1d(0,1,2)

Grid 1d : début 0.0 , fin 1.0 , 2 éléments


In [37]:
a+2

Grid 1d : début 0.0 , fin 1.0 , 4 éléments


In [38]:
a += 1

Grid 1d : début 0.0 , fin 1.0 , 5 éléments


Attention l'addition n'est pas forcément commutative !

In [39]:
try 
    2 + a
catch e
    showerror(stdout, e)
end

MethodError: no method matching +(::Int64, ::Grid1d)
Closest candidates are:
  +(::Any, ::Any, ::Any, ::Any...) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/operators.jl:655
  +(::T, ::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/int.jl:87
  +(::Union{Int16, Int32, Int64, Int8}, ::BigInt) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/gmp.jl:535
  ...

ni unaire !

In [40]:
try 
    -a
catch e
    showerror(stdout, e)
end 

MethodError: no method matching -(::Grid1d)
Closest candidates are:
  -(::Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8}) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/int.jl:85
  -(::T, ::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/int.jl:86
  -(::Union{UInt16, UInt32, UInt64, UInt8}, ::BigInt) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/gmp.jl:532
  ...

Notez le message d'erreur qui est très claire !

In [41]:
try
    a+[1,2]
catch e
    showerror(stdout, e)
end 

MethodError: no method matching +(::Grid1d, ::Vector{Int64})
Closest candidates are:
  +(::Any, ::Any, ::Any, ::Any...) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/operators.jl:655
  +(::SparseArrays.AbstractSparseMatrixCSC, ::Array) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/SparseArrays/src/sparsematrix.jl:1774
  +(::Array, ::Array...) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/arraymath.jl:43
  ...

## Autres surcharges

Toutes les fonctions usuelles sont surchargeable sans limite : size(); det() ...

In [42]:
function Base.size(g::AbstractGrid)
    return g.n
end

In [43]:
size(a)

5

In [44]:
using LinearAlgebra

function LinearAlgebra.det(g::Grid1d)
    g.fin-g.debut
end 

In [45]:
det(a)

1.0

# Type et constructeurs

Chaque langage "objet" définit un constructeur pour ces objets. Nous avons déjà utilisé un constructeur générique qui rempli chaque champ du nouveau type. Il est possible de faire une variante suivant le nombre d'arguments d'entrée et de leur type 

In [46]:
abstract type AbstractGrid end # juste en dessous de Any

struct Grid1D <: AbstractGrid
    debut::Float64
    fin::Float64
    n::Int32
    
    # constructeurs par défaut sans argument
    function Grid1D()
        new(0,0,0)
    end
    
    # constructeurs par défaut avec argument
    function Grid1D(a,b,c)
        c <= 0 && @error("pas possible")
        new(a,b,c)
    end
end

In [48]:
b = Grid1D(0, 1, -1)

┌ Error: pas possible
└ @ Main In[46]:15


Grid1D(0.0, 1.0, -1)

Il devient possible de déterminer un constructeurs pour différentes entrées.

Il faut au préalable bien penser sa hiérarchie de type et écrire autant de fonctions constructeurs que de cas d'initialisation du nouveau type.

# Les Itérateurs

Il est possible sur un type nouveau de définir un itérateur, comme ici de parcourir les points de la grille, définissons (surchargeons) de nouvelles fonctions ou plutôt méthodes : 

In [50]:
Base.iterate(g::Grid1D, state=g.debut) = begin
   state > g.fin ? nothing : (state, state+(g.fin-g.debut)/g.n)
end

In [49]:
grid = Grid1D(0, 2 , 10)

Grid1D(0.0, 2.0, 10)

In [51]:
for x in grid
    println(x)
end

0.0
0.2
0.4
0.6000000000000001
0.8
1.0
1.2
1.4
1.5999999999999999
1.7999999999999998
1.9999999999999998


Il devient possible de construire des itérateurs sur une grille 2d, 3d renvoyant les coordonnées des points de la grille... Mais on peut imaginer sur une triangulation etc... 